In [1]:
import pandas as pd

### Reading the data

In [5]:
df = pd.read_csv('../data/ecommerceDatasetNLP.csv', header=None)
df.columns = ['label', 'text']
df.head()

,label,text
0,Household,Paper Plane Design Framed Wall Hanging Motivat...
1,Household,"SAF 'Floral' Framed Painting (Wood, 30 inch x ..."
2,Household,SAF 'UV Textured Modern Art Print Framed' Pain...
3,Household,"SAF Flower Print Framed Painting (Synthetic, 1..."
4,Household,Incredible Gifts India Wooden Happy Birthday U...


In [6]:
df['label'].value_counts()

label
Household                 19313
Books                     11820
Electronics               10621
Clothing & Accessories     8671
Name: count, dtype: int64

In [7]:
df.isnull().sum()

label    0
text     1
dtype: int64

In [8]:
df.dropna(inplace=True)
df.isnull().sum()

label    0
text     0
dtype: int64

In [16]:
df.shape

(50424, 2)

In [22]:
print(df[df['label'] == 'Electronics']['text'].iloc[0])

Dell 19.5V-3.34AMP 65W Laptop Adapter (Without power Cord) Design Features of Dell Laptop - Power adapter - 65 Watt (No Power Cable) The Dell 65-Watt Inspiron AC Power Adapter helps you to charge Dell laptops, with screen size ranging from 13-inches to 17-inches. The Inspiron power adapter provides you with continuous power supply. The adapter is lightweight for ease of use. The 3-pronged plug design of the Dell Inspiron laptop adapter prevents voltage drop. You can easily plug it into any wall outlet to deliver AC power to your laptop


### Text Preprocessing

In [23]:
from nltk import word_tokenize
from nltk.corpus import stopwords

STOP_WORDS = set(stopwords.words('english'))


In [24]:
def word_tokenization(text):
    words = word_tokenize(text)
    return words


def remove_stopwords(text):
    words = word_tokenization(text)
    for stop_word in STOP_WORDS:
        if stop_word in words:
            words.remove(stop_word)
    return words

In [25]:
def clean_text(text):
    tokens1 = remove_stopwords(text)
    return tokens1

### Tf-IDF Vectorization

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(analyzer=clean_text)
X_train = tfidf.fit_transform(df['text'])